# Save and load models
Model can resume training where it left off to avoid long training times. You can also share your model with others. Typically these things are shared:
- code to create the model
- the model's trained weights/parameters

#### Note
There is more than one way of going about this.

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.8.0


Using the MNIST dataset, however, only using the first 1,000 values

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 1s 0us/step


In [3]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation="relu", input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2022-06-12 17:53:23.500903: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


We're going to create a callback that saves the weights only during training

In [4]:
checkpoint_path = "training_1/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

Now that we've set the model to save, we're going to train a new model

In [5]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[ckpt_callback])

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.3203 - sparse_categorical_accuracy: 0.6178
Epoch 1: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 1.2092 - sparse_categorical_accuracy: 0.6540 - val_loss: 0.7684 - val_sparse_categorical_accuracy: 0.7490
Epoch 2/10
21/32 [==================>...........] - ETA: 0s - loss: 0.4611 - sparse_categorical_accuracy: 0.8735
Epoch 2: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.4376 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5342 - val_sparse_categorical_accuracy: 0.8320
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2907 - sparse_categorical_accuracy: 0.9271
Epoch 3: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 0.2876 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4818 - val_sparse_categorical_accuracy: 0.8410
Ep

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'checkpoint.ckpt.index', 'checkpoint.ckpt.data-00000-of-00001']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

This is slightly odd, but we're intentionally creating an untrained model and evaluating its accuracy.

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Untrained model accuracy {100 * acc}%")

32/32 - 0s - loss: 2.2714 - sparse_categorical_accuracy: 0.1750 - 137ms/epoch - 4ms/step
Untrained model accuracy 17.499999701976776%


Loading the weights from our previous training session and testing again

In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Untrained model accuracy {100 * acc}%")

32/32 - 0s - loss: 0.4036 - sparse_categorical_accuracy: 0.8730 - 60ms/epoch - 2ms/step
Untrained model accuracy 87.30000257492065%


There are various ways to get unique names, and you can manupilate the saving frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [9]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5*batch_size
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels, epochs=50, batch_size=batch_size, callbacks=[ckpt_callback] ,validation_data=(test_images, test_labels), verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

training_2/cp-0050.ckpt


Testing this by resetting the model and loading the latest checkpoint.

In [12]:
model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print(f"Restored model accuracy {acc * 100:5.2f}%")

32/32 - 0s - loss: 0.4751 - sparse_categorical_accuracy: 0.8730 - 166ms/epoch - 5ms/step
Restored model accuracy 87.30%


### What are these files?
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
- one or more "shards" that contain the model's weights
- an index file that indicates which weights are stored in which shard

If you are training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

### Saving weights manually
By default the TensorFlow checkpoint format (`.ckpt` extension) is used.

In [14]:
model.save_weights('./checkpoints/my_checkpoint')

# Reset and test
model = create_model()

model.load_weights("./checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model accuracy {acc*100:5.2f}%")

32/32 - 0s - loss: 0.4751 - sparse_categorical_accuracy: 0.8730 - 148ms/epoch - 5ms/step
Restored model accuracy 87.30%


## Saving the entire model
Call `model.save` to save a model's architecture, weights, and training configuration in a single file/folder. Can either be in TensorFlow's `SavedModel` format or `HDF5`, `SavedModel` is default in TF2.x.

You can use both these formats to load into TensorFlow.js (still unclear why you'd train here... but you can) or TensorFlow Lite.

#### SavedModel format
This is the default format

In [15]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.1341 - sparse_categorical_accuracy: 0.6750
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4364 - sparse_categorical_accuracy: 0.8710
Epoch 3/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2961 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2156 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1703 - sparse_categorical_accuracy: 0.9590


2022-06-12 18:36:40.347073: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [17]:
!ls saved_model
!ls saved_model/my_model

my_model
assets            keras_metadata.pb saved_model.pb    variables


The python code from earlier is not needed and the model can be loaded from nothing. 

In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model accuracy: {acc*100:5.2f}")

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4490 - sparse_categorical_accuracy: 0.8600 - 73ms/epoch - 2ms/step
Restored model accuracy: 86.00
(1000, 10)


#### HDF5 format

In [21]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1458 - sparse_categorical_accuracy: 0.6680
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4123 - sparse_categorical_accuracy: 0.8940
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2778 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2059 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1664 - sparse_categorical_accuracy: 0.9640


In [22]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model accuracy: {acc*100:5.2f}")

32/32 - 0s - loss: 0.4462 - sparse_categorical_accuracy: 0.8580 - 340ms/epoch - 11ms/step
Restored model accuracy: 85.80


Keras saves pretty much everything, however, is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints.

### Custom objects

There are some extra steps to save custom objects in HDF5. [link](https://www.tensorflow.org/tutorials/keras/save_and_load#saving_custom_objects)